In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gmean
from pandas import DataFrame


In [28]:
df_Results = pd.read_pickle("UsableData/Results.pkl")
df_Results.head(5)

,Market Cap,Employees,Book Value,Equal,Random,Variance,Custom
1995-12-31,1,1,1,1,1,1,1
1996-01-31,1.055732,1.037344,1.027627,1.00793,1.007622,0.975108,1.022483
1996-02-29,1.06463,1.050292,1.042715,1.027357,1.027117,1.001764,1.038705
1996-03-31,1.036932,1.029997,1.026332,1.018903,1.018787,1.006524,1.024392
1996-04-30,1.043941,1.036461,1.032508,1.024496,1.024371,1.011145,1.030416


In [29]:
# Based on 3.1 Finance Introduction
total_cumulative_return = (df_Results.iloc[-1]-df_Results.iloc[1])*100
total_cumulative_return

Market Cap    422.333998
Employees     643.160159
Book Value    517.509186
Equal         555.230327
Random        572.228095
Variance      113.853416
Custom        624.791655
dtype: object

In [30]:
# Based on 3.1 Finance Introduction
mean_yearly_return = (((1+(total_cumulative_return/100)))**(1/10)-1)*100
mean_yearly_return

Market Cap    17.976316
Employees     22.210421
Book Value    19.967703
Equal         20.681139
Random         20.99061
Variance       7.897559
Custom        21.904944
dtype: object

In [31]:
df_Returns_Strat = df_Results.pct_change()*100
df_Returns_Strat.dropna(axis=0, inplace=True)
df_Returns_Strat.head(5)

,Market Cap,Employees,Book Value,Equal,Random,Variance,Custom
1996-01-31,5.573161,3.734411,2.762652,0.793015,0.762178,-2.489193,2.248295
1996-02-29,0.842857,1.248215,1.468301,1.927407,1.934737,2.733667,1.586487
1996-03-31,-2.601658,-1.932360,-1.571210,-0.822860,-0.810965,0.475171,-1.377918
1996-04-30,0.675964,0.627619,0.601806,0.548916,0.548082,0.459045,0.588068
1996-05-31,9.309816,9.269007,9.247201,9.202487,9.201782,9.126401,9.235592


In [32]:
monthly_vol = np.std(df_Returns_Strat, axis=0)
monthly_vol

Market Cap    5.685149
Employees     5.872179
Book Value    5.941810
Equal         4.492707
Random        4.510380
Variance      5.026623
Custom        4.982646
dtype: float64

In [33]:
# Read yearly return of DE 10 Year Bound reported monthly
DE10Y = pd.read_csv("RawData/DE10Y.csv", index_col="Datum")
DE10Y.index = pd.to_datetime(DE10Y.index, dayfirst=True)
DE10Y = DE10Y[(DE10Y.index >= df_Results.index[1]) & (DE10Y.index <= df_Results.index[-1])]
for row in DE10Y.index:
    string = DE10Y.loc[row, "Zuletzt"].strip("%").replace(".", "").replace(",", ".")
    DE10Y.at[row, "Zuletzt"] = float(string)
DE10Y.head(5)

,Zuletzt,Eröffn.,Hoch,Tief,+/- %
Datum,,,,,
2019-12-01,-0.187,"-0,3230","-0,1720","-0,3650","-48,06%"
2019-11-01,-0.36,"-0,4070","-0,2180","-0,4170","-10,67%"
2019-10-01,-0.403,"-0,5490","-0,3200","-0,6040","-29,55%"
2019-09-01,-0.572,"-0,7030","-0,4340","-0,7430","-18,63%"
2019-08-01,-0.703,"-0,4330","-0,4120","-0,7280","59,05%"


In [34]:
# Based on 3.1 Finance Introduction
risk_free_return = np.mean(DE10Y["Zuletzt"])
risk_free_return

3.047656794425087

In [35]:
# Based on 3.1 Finance Introduction
sharpe_ratio = pd.Series(index=df_Results.columns)
for strategy in mean_yearly_return.index:
    sharpe_ratio[strategy] = (mean_yearly_return[strategy] - risk_free_return) / monthly_vol[strategy]
sharpe_ratio

Market Cap    2.625905
Employees     3.263314
Book Value    2.847625
Equal         3.924913
Random        3.978147
Variance      0.964843
Custom        3.784593
dtype: float64

In [36]:
# maximum monthly drawdown
max_monthly_drawdown = pd.Series(index=df_Results.columns)
for strategy in df_Returns_Strat.columns:
    max_monthly_drawdown[strategy] = min(df_Returns_Strat[strategy])
max_monthly_drawdown

Market Cap   -22.348892
Employees    -24.473774
Book Value   -22.915366
Equal        -15.604835
Random       -16.063150
Variance     -27.235294
Custom       -20.107138
dtype: float64

In [37]:
positive_month = pd.Series(index=df_Results.columns)
for strategy in df_Returns_Strat.columns:
    returns = df_Returns_Strat[strategy]
    returns = returns[returns > 0]
    positive_month[strategy] = len(returns)
positive_month

Market Cap    170.0
Employees     175.0
Book Value    172.0
Equal         181.0
Random        183.0
Variance      174.0
Custom        172.0
dtype: float64

In [38]:
df_Analysis = DataFrame(index=df_Results.columns)
df_Analysis["Total Cumulative Return"] = total_cumulative_return
df_Analysis["Mean Yearly Return"] = mean_yearly_return
df_Analysis["Volatility"] = monthly_vol
df_Analysis["Sharpe Ratio"] = sharpe_ratio
df_Analysis["Max Drawdown"] = max_monthly_drawdown
df_Analysis["Positive Month"] = positive_month
df_Analysis.to_pickle("UsableData/Analysis.pkl")
df_Analysis

,Total Cumulative Return,Mean Yearly Return,Volatility,Sharpe Ratio,Max Drawdown,Positive Month
Market Cap,422.333998,17.976316,5.685149,2.625905,-22.348892,170.0
Employees,643.160159,22.210421,5.872179,3.263314,-24.473774,175.0
Book Value,517.509186,19.967703,5.941810,2.847625,-22.915366,172.0
Equal,555.230327,20.681139,4.492707,3.924913,-15.604835,181.0
Random,572.228095,20.99061,4.510380,3.978147,-16.063150,183.0
Variance,113.853416,7.897559,5.026623,0.964843,-27.235294,174.0
Custom,624.791655,21.904944,4.982646,3.784593,-20.107138,172.0
